# Wind Downscaling

## Prerequisites

Get a Copernicus API key from: https://cds.climate.copernicus.eu/api-how-to

In [ ]:
!pip install -e .

In [ ]:
!pip install -U git+https://github.com/OpheliaMiralles/WindDownscaling_EPFL_UNIBE.git

In [ ]:
!pip install xarray numpy pandas pysftp cdsapi topo-descriptors elevation rasterio dask python-dotenv

In [ ]:
!conda install -y -c conda-forge gdal

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow-gpu

In [ ]:
from datetime import date
from pathlib import Path
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
DATA_ROOT = Path('./data')
ERA5_DATA_FOLDER = DATA_ROOT / 'ERA5'
COSMO1_DATA_FOLDER = DATA_ROOT / 'COSMO1'
DEM_DATA_FILE = DATA_ROOT / 'dem/Switzerland-90m-DEM.tif'
PROCESSED_DATA_FOLDER = DATA_ROOT / 'img_prediction_files'

DATA_ROOT.mkdir(parents=True, exist_ok=True)
ERA5_DATA_FOLDER.mkdir(exist_ok=True)
COSMO1_DATA_FOLDER.mkdir(exist_ok=True)
DEM_DATA_FILE.parent.mkdir(exist_ok=True)
PROCESSED_DATA_FOLDER.mkdir(exist_ok=True)

In [ ]:
START_DATE = date(2017, 9, 1)
END_DATE = date(2017, 9, 3)
SEQUENCE_LENGTH = 6
IMG_SIZE = 30
BATCH_SIZE = 16

In [ ]:
from data.download_ERA5 import download_ERA5
download_ERA5(ERA5_DATA_FOLDER, START_DATE, END_DATE)
print('Done')

In [ ]:
if not DEM_DATA_FILE.exists():
    dest = str(DEM_DATA_FILE)
    !eio --product SRTM3 clip -o {dest} --bounds 5.27 45.46 11.02 48.15

In [ ]:
from data.data_processing import process_topographic_variables_file
process_topographic_variables_file(DEM_DATA_FILE)

In [ ]:
username = os.getenv('COSMO_USERNAME')
password = os.getenv('COSMO_PASSWORD')
from data import download_COSMO1
download_COSMO1(username, password, COSMO1_DATA_FOLDER, START_DATE, END_DATE)

In [ ]:
from data.data_processing import process_imgs
process_imgs(PROCESSED_DATA_FOLDER, ERA5_DATA_FOLDER, COSMO1_DATA_FOLDER, DEM_DATA_FILE.parent,
             START_DATE, END_DATE)
print('Done')

In [ ]:
from gan.data_generator import BatchGenerator, NaiveDecoder
batches = BatchGenerator(PROCESSED_DATA_FOLDER, NaiveDecoder(), SEQUENCE_LENGTH, IMG_SIZE, BATCH_SIZE)
x, y = next(batches)
x.shape, y.shape

In [ ]:
from gan.train import build_gan
gan = build_gan(n_inputs=len(batches.input_variables), n_timesteps=SEQUENCE_LENGTH, gen_disc_ratio=8,
                low_res_size=(IMG_SIZE, IMG_SIZE), high_res_size=(IMG_SIZE, IMG_SIZE))

In [ ]:
gan.train(batches, 1, 1, verbose=True)